In [1]:
#importing required Libraries
import pandas as pd   
import requests       
from bs4 import BeautifulSoup 
import numpy as np 

In [2]:
url = 'https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating'
#request allow you to send HTTP request
response = requests.get(url)
print(response)
soup = BeautifulSoup(response.content, 'html.parser')

<Response [200]>


In [3]:
movie_name = []
year = []
time = []
rating = []
metascore = []
votes = []
gross = []

description = []
Director = []
Stars = []

In [4]:
#storing the meaningfull required data in the variable
movie_data = soup.findAll('div', attrs= {'class': 'lister-item mode-advanced'})

In [5]:
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36'}


# import requests
# from bs4 import BeautifulSoup

# for store in movie_data:
    
#     div_element = store.find('div', {'class': 'ribbonize'})
#     movie_id = div_element.get('data-tconst')

#     url = 'https://www.imdb.com/title/' + movie_id + '/?ref_=adv_li_tt'
#     response = requests.get(url,headers=headers)
#     html_content = response.text
#     soup1 = BeautifulSoup(html_content, 'html.parser')
    
#     # Find the div containing the star cast information
#     star_cast_div = soup1.find_all('a', class_='sc-bfec09a1-1 fUguci')

#     # Extract the star cast names
#     star_cast_names = []
#     for link in star_cast_div:
#         star_cast_names.append(link.text)

#     Stars.append(star_cast_names)

# print(Stars)
    

In [6]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36'}

#calling one by one using for loop
for store in movie_data:  
    
    name = store.h3.a.text
    movie_name.append(name)
       
    year_of_release = store.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.replace('(', '').replace(')', '')
    year.append(year_of_release)
    
    runtime = store.p.find('span', class_ = 'runtime').text.replace(' min', '')
    time.append(runtime)
    
    rate = store.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')
    rating.append(rate)
    
    meta  = store.find('span', class_ = 'metascore').text.replace(' ', '') if store.find('span', class_ = 'metascore') else '^^^^^^'
    metascore.append(meta)
    #since, gross and votes have same attributes, that's why we had created a common variable and then used indexing
    value = store.find_all('span', attrs = {'name': 'nv'})
    
    vote = value[0].text
    votes.append(vote)
    
    grosses = value[1].text if len(value) >1 else '*****'
    gross.append(grosses)
    
    # Description of the Movies -- Not explained in the Video, But you will figure it out. 
    describe = store.find_all('p', class_ = 'text-muted')
    description_ = describe[1].text.replace('\n', '') if len(describe) >1 else '*****'
    description.append(description_)
    
    #Cast Details -- Scraping Director name and Stars -- Not explained in Video
    cast = store.find("p", class_ = '')
    cast = cast.text.replace('\n', '').split('|')
    cast = [x.strip() for x in cast]
    cast = [cast[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
    Director.append(cast[0])
#     Stars.append([x.strip() for x in cast[1].split(",")])

    div_element = store.find('div', {'class': 'ribbonize'})
    movie_id = div_element.get('data-tconst')

    url = 'https://www.imdb.com/title/' + movie_id + '/?ref_=adv_li_tt'
    response = requests.get(url,headers=headers)
    html_content = response.text
    soup1 = BeautifulSoup(html_content, 'html.parser')
    
    # Find the div containing the star cast information
    star_cast_div = soup1.find_all('a', class_='sc-bfec09a1-1 fUguci')

    # Extract the star cast names
    star_cast_names = []
    for link in star_cast_div:
        star_cast_names.append(link.text)

    Stars.append(star_cast_names)
    
    

In [7]:
#creating a dataframe using pandas library
movie_DF = pd.DataFrame({'Name of movie': movie_name, 'Year of relase': year, 'Watchtime': time, 'Movie Rating': rating, 'Metascore': metascore, 'Votes': votes, 'Gross collection': gross, 'Description': description, "Director": Director, 'Star': Stars})
movie_DF

,Name of movie,Year of relase,Watchtime,Movie Rating,Metascore,Votes,Gross collection,Description,Director,Star
0,The Shawshank Redemption,1994,142,9.3,82,"2,747,657",$28.34M,"Over the course of several years, two convicts...",Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will..."
1,The Godfather,1972,175,9.2,100,"1,911,065",$134.97M,"Don Vito Corleone, head of a mafia family, dec...",Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan, Diane K..."
2,Spider-Man: Across the Spider-Verse,2023,140,9.1,86,"56,773",#15,"Miles Morales catapults across the Multiverse,...","Directors:Joaquim Dos Santos, Kemp Powers, Jus...","[Shameik Moore, Hailee Steinfeld, Brian Tyree ..."
3,The Dark Knight,2008,152,9.0,84,"2,720,357",$534.86M,When the menace known as the Joker wreaks havo...,Christopher Nolan,"[Christian Bale, Heath Ledger, Aaron Eckhart, ..."
4,Schindler's List,1993,195,9.0,95,"1,385,960",$96.90M,"In German-occupied Poland during World War II,...",Steven Spielberg,"[Liam Neeson, Ralph Fiennes, Ben Kingsley, Car..."
...,...,...,...,...,...,...,...,...,...,...
95,Scarface,1983,170,8.3,65,"866,805",$45.60M,"In 1980 Miami, a determined Cuban immigrant ta...",Brian De Palma,"[Al Pacino, Michelle Pfeiffer, Steven Bauer, M..."
96,American Beauty,1999,122,8.3,84,"1,174,028",$130.10M,A sexually frustrated suburban father has a mi...,Sam Mendes,"[Kevin Spacey, Annette Bening, Thora Birch, We..."
97,North by Northwest,1959,136,8.3,98,"334,769",$13.28M,A New York City advertising executive goes on ...,Alfred Hitchcock,"[Cary Grant, Eva Marie Saint, James Mason, Jes..."
98,2001: A Space Odyssey,1968,149,8.3,84,"685,889",$56.95M,After uncovering a mysterious artifact buried ...,Stanley Kubrick,"[Keir Dullea, Gary Lockwood, William Sylvester..."


In [8]:
#Saving data in Excel file:

movie_DF.to_excel("100_IMDB_Movies_Data.xlsx")
